<a href="https://colab.research.google.com/github/jan-kreischer/UZH_ML4NLP/blob/main/Project-02/ex02_wordembeddings_jan.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Project 2 - Word Embeddings with PyTorch

## 1. Setup
### 1.1 Imports

In [1]:

# Author: Jan Kreischer Bauer
import os

# torch
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
torch.manual_seed(1)

# numpy and pandas
import numpy as np
import pandas as pd
pd.set_option('max_colwidth', 800)

import regex as re

# tokenization
import nltk;
nltk.download('stopwords');
%matplotlib inline

from argparse import Namespace
from collections import Counter
import json
import os
import re
import string
import itertools
import numpy as np
import pandas as pd
import torch
import tqdm
from torch.utils.data.sampler import SubsetRandomSampler
CUDA = torch.cuda.is_available()
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu:0')
torch.cuda.set_device(device)
print('Using device:', device)
import torch.nn as nn
from torch.autograd import Variable
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm_notebook
from sklearn.model_selection import train_test_split

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
Using device: cuda:0


### 1.2 Environment

In [2]:
# Check GPU
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Sat Oct 23 07:14:59 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.74       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   47C    P0    29W / 250W |      2MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [177]:
!nvidia-smi -L

GPU 0: Tesla P100-PCIE-16GB (UUID: GPU-ac895e14-ddd0-b216-862c-49f315224518)


In [3]:
# Check Memory
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 27.3 gigabytes of available RAM

You are using a high-RAM runtime!


### 1.3 Constants

In [48]:
FEATURE_COLUMN = 'Review'
CONTEXT_OFFSET = 2 # n words to the left, n to the right
BATCH_SIZE = 64

EPOCHS_HOTEL = 12
EMBEDDING_DIM_HOTEL = 50

EPOCHS_SCIFI = 2
EMBEDDING_DIM_HOTEL = 50

Source: [https://pytorch.org/tutorials/beginner/nlp/word_embeddings_tutorial.html#exercise-computing-word-embeddings-continuous-bag-of-words](https://pytorch.org/tutorials/beginner/nlp/word_embeddings_tutorial.html#exercise-computing-word-embeddings-continuous-bag-of-words)

# Word Embeddings: Encoding Lexical Semantics

Word embeddings are dense vectors of real numbers, one per word in your
vocabulary. In NLP, it is almost always the case that your features are
words! But how should you represent a word in a computer? You could
store its ascii character representation, but that only tells you what
the word *is*, it doesn't say much about what it *means* (you might be
able to derive its part of speech from its affixes, or properties from
its capitalization, but not much). Even more, in what sense could you
combine these representations? We often want dense outputs from our
neural networks, where the inputs are $|V|$ dimensional, where
$V$ is our vocabulary, but often the outputs are only a few
dimensional (if we are only predicting a handful of labels, for
instance). How do we get from a massive dimensional space to a smaller
dimensional space?

How about instead of ascii representations, we use a one-hot encoding?
That is, we represent the word $w$ by

\begin{align}\overbrace{\left[ 0, 0, \dots, 1, \dots, 0, 0 \right]}^\text{|V| elements}\end{align}

where the 1 is in a location unique to $w$. Any other word will
have a 1 in some other location, and a 0 everywhere else.

There is an enormous drawback to this representation, besides just how
huge it is. It basically treats all words as independent entities with
no relation to each other. What we really want is some notion of
*similarity* between words. Why? Let's see an example.

Suppose we are building a language model. Suppose we have seen the
sentences

* The mathematician ran to the store.
* The physicist ran to the store.
* The mathematician solved the open problem.

in our training data. Now suppose we get a new sentence never before
seen in our training data:

* The physicist solved the open problem.

Our language model might do OK on this sentence, but wouldn't it be much
better if we could use the following two facts:

* We have seen  mathematician and physicist in the same role in a sentence. Somehow they
  have a semantic relation.
* We have seen mathematician in the same role  in this new unseen sentence
  as we are now seeing physicist.

and then infer that physicist is actually a good fit in the new unseen
sentence? This is what we mean by a notion of similarity: we mean
*semantic similarity*, not simply having similar orthographic
representations. It is a technique to combat the sparsity of linguistic
data, by connecting the dots between what we have seen and what we
haven't. This example of course relies on a fundamental linguistic
assumption: that words appearing in similar contexts are related to each
other semantically. This is called the `distributional
hypothesis <https://en.wikipedia.org/wiki/Distributional_semantics>`__.


# Getting Dense Word Embeddings

How can we solve this problem? That is, how could we actually encode
semantic similarity in words? Maybe we think up some semantic
attributes. For example, we see that both mathematicians and physicists
can run, so maybe we give these words a high score for the "is able to
run" semantic attribute. Think of some other attributes, and imagine
what you might score some common words on those attributes.

If each attribute is a dimension, then we might give each word a vector,
like this:

\begin{align}q_\text{mathematician} = \left[ \overbrace{2.3}^\text{can run},
   \overbrace{9.4}^\text{likes coffee}, \overbrace{-5.5}^\text{majored in Physics}, \dots \right]\end{align}

\begin{align}q_\text{physicist} = \left[ \overbrace{2.5}^\text{can run},
   \overbrace{9.1}^\text{likes coffee}, \overbrace{6.4}^\text{majored in Physics}, \dots \right]\end{align}

Then we can get a measure of similarity between these words by doing:

\begin{align}\text{Similarity}(\text{physicist}, \text{mathematician}) = q_\text{physicist} \cdot q_\text{mathematician}\end{align}

Although it is more common to normalize by the lengths:

\begin{align}\text{Similarity}(\text{physicist}, \text{mathematician}) = \frac{q_\text{physicist} \cdot q_\text{mathematician}}
   {\| q_\text{\physicist} \| \| q_\text{mathematician} \|} = \cos (\phi)\end{align}

Where $\phi$ is the angle between the two vectors. That way,
extremely similar words (words whose embeddings point in the same
direction) will have similarity 1. Extremely dissimilar words should
have similarity -1.


You can think of the sparse one-hot vectors from the beginning of this
section as a special case of these new vectors we have defined, where
each word basically has similarity 0, and we gave each word some unique
semantic attribute. These new vectors are *dense*, which is to say their
entries are (typically) non-zero.

But these new vectors are a big pain: you could think of thousands of
different semantic attributes that might be relevant to determining
similarity, and how on earth would you set the values of the different
attributes? Central to the idea of deep learning is that the neural
network learns representations of the features, rather than requiring
the programmer to design them herself. So why not just let the word
embeddings be parameters in our model, and then be updated during
training? This is exactly what we will do. We will have some *latent
semantic attributes* that the network can, in principle, learn. Note
that the word embeddings will probably not be interpretable. That is,
although with our hand-crafted vectors above we can see that
mathematicians and physicists are similar in that they both like coffee,
if we allow a neural network to learn the embeddings and see that both
mathematicians and physicists have a large value in the second
dimension, it is not clear what that means. They are similar in some
latent semantic dimension, but this probably has no interpretation to
us.


In summary, **word embeddings are a representation of the *semantics* of
a word, efficiently encoding semantic information that might be relevant
to the task at hand**. You can embed other things too: part of speech
tags, parse trees, anything! The idea of feature embeddings is central
to the field.


# Word Embeddings in Pytorch

Before we get to a worked example and an exercise, a few quick notes
about how to use embeddings in Pytorch and in deep learning programming
in general. Similar to how we defined a unique index for each word when
making one-hot vectors, we also need to define an index for each word
when using embeddings. These will be keys into a lookup table. That is,
embeddings are stored as a $|V| \times D$ matrix, where $D$
is the dimensionality of the embeddings, such that the word assigned
index $i$ has its embedding stored in the $i$'th row of the
matrix. In all of my code, the mapping from words to indices is a
dictionary named word\_to\_ix.

The module that allows you to use embeddings is torch.nn.Embedding,
which takes two arguments: the vocabulary size, and the dimensionality
of the embeddings.

To index into this table, you must use torch.LongTensor (since the
indices are integers, not floats).




# Exercise: Computing Word Embeddings: Continuous Bag-of-Words

The Continuous Bag-of-Words model (CBOW) is frequently used in NLP deep
learning. It is a model that tries to predict words given the context of
a few words before and a few words after the target word. This is
distinct from language modeling, since CBOW is not sequential and does
not have to be probabilistic. Typcially, CBOW is used to quickly train
word embeddings, and these embeddings are used to initialize the
embeddings of some more complicated model. Usually, this is referred to
as *pretraining embeddings*. It almost always helps performance a couple
of percent.

The CBOW model is as follows. Given a target word $w_i$ and an
$N$ context window on each side, $w_{i-1}, \dots, w_{i-N}$
and $w_{i+1}, \dots, w_{i+N}$, referring to all context words
collectively as $C$, CBOW tries to minimize

\begin{align}-\log p(w_i | C) = -\log \text{Softmax}(A(\sum_{w \in C} q_w) + b)\end{align}

where $q_w$ is the embedding for word $w$.

Implement this model in Pytorch by filling in the class below. Some
tips:

* Think about which parameters you need to define.
* Make sure you know what shape each operation expects. Use .view() if you need to
  reshape.




## 2. Data Preprocessing
### 2.1 Data Acquisition

In [5]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [6]:
import os
os.chdir("/content/gdrive/My Drive/ML4NLP/Projects/Project-02/")
!ls

current_version.ipynb
ex02_task.pdf
ex02_wordembeddings_hongjie.ipynb
ex02_wordembeddings_hongjie.ipynbex02_wordembeddings_hongjie.ipynb
ex02_wordembeddings_jan.ipynb
Intro_to_PyTorch.ipynb
old
README.md
scifi.txt
tripadvisor_hotel_reviews.csv


In [7]:
reviews_dataset = pd.read_csv('./tripadvisor_hotel_reviews.csv')

In [8]:
reviews_dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20491 entries, 0 to 20490
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Review  20491 non-null  object
 1   Rating  20491 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 320.3+ KB


### 2.2 Data Cleaning

In [9]:
wpt = nltk.WordPunctTokenizer()
stop_words = nltk.corpus.stopwords.words('english')

def clean_document(x):
    x = re.sub(r'[^a-zA-Z\s]', '', x.lower(), re.I|re.A)
    x = re.sub(r'\w*\d\w*', '', x)
    x = re.sub(r'[\-!+_@*#\/$:)"\'.;,?&({}[]]*', '', x)
    x = x.strip()
    tokens = wpt.tokenize(x)
    filtered_tokens = [token for token in tokens if token not in stop_words]
    x = ' '.join(filtered_tokens)
    return x

clean_corpus = np.vectorize(clean_document)

In [10]:
reviews = clean_corpus(reviews_dataset['Review'])
reviews[:10]

array(['nice hotel expensive parking got good deal stay hotel anniversary arrived late evening took advice previous reviews valet parking check quick easy little disappointed nonexistent view room room clean nice size bed comfortable woke stiff neck high pillows soundproof like heard music room night morning loud bangs doors opening closing hear people talking hallway maybe noisy neighbors aveda bath products nice goldfish stay nice touch taken advantage staying longer location great walking distance shopping overall nice experience pay parking night',
       'ok nothing special charge diamond member hilton decided chain shot th anniversary seattle start booked suite paid extra website description suite bedroom bathroom standard hotel room took printed reservation desk showed said things like tv couch ect desk clerk told oh mixed suites description kimpton website sorry free breakfast got kidding embassy suits sitting room bathroom bedroom unlike kimpton calls suite day stay offer corr

In [11]:
# Since we want to train a CBOW model with context width of 2
# on the reviews, we drop all reviews with less than 5 words.
# This is equivalent to only keeping instances with at least 5 words.
reviews = [review for review in reviews if len(review.split(" ")) >=  (2*CONTEXT_OFFSET + 1)]

In [12]:
# In order to build the corpus for the reviews 
# we want to find every distinct word that occurs
# in at least one review.
# We join all reviews into one large string and then
# split it at every space to receive a list of words
# Then the set method is used in order to only
# retain unique words.
# This list is then alphabetically sorted
reviews_vocabulary = sorted(set((" ".join(reviews)).split()))

In [13]:
reviews_vocabulary_size = len(reviews_vocabulary)
print("The reviews use a vocabulary comprising {} different words.".format(reviews_vocabulary_size))

The reviews use a vocabulary comprising 75255 different words.


In [14]:
word2index = {w:i for i,w in enumerate(reviews_vocabulary)} # Lookup table mapping words to indices
index2word = {i:w for i,w in enumerate(reviews_vocabulary)} # Lookup table mapping indices to words

In [15]:
data = []
for review in reviews:
  raw_text = review.split()
  for i in range(CONTEXT_OFFSET, len(raw_text) - CONTEXT_OFFSET):
      context = [raw_text[i - 2], raw_text[i - 1],
                raw_text[i + 1], raw_text[i + 2]]
      #print(context)
      target = raw_text[i]
      data.append((context, target))
print(data[:5])

[(['nice', 'hotel', 'parking', 'got'], 'expensive'), (['hotel', 'expensive', 'got', 'good'], 'parking'), (['expensive', 'parking', 'good', 'deal'], 'got'), (['parking', 'got', 'deal', 'stay'], 'good'), (['got', 'good', 'stay', 'hotel'], 'deal')]


In [16]:
# create your model and train.  here are some functions to help you make
# the data ready for use by your module
def make_context_vector(context, word2index):
    idxs = [word2index[w] for w in context]
    return torch.tensor(idxs, dtype=torch.long)

make_context_vector(data[0][0], word2index)  # example

tensor([43815, 31861, 47687, 28624])

In [162]:
class HotelReviewsDataset(Dataset):
  def __init__(self, X, y):
    self.X = X
    self.y = y

  def __getitem__(self, idx):
    return self.X[idx], self.y[idx]

  def __len__(self):
    return len(self.X)

In [18]:
X = np.array([i[0] for i in data])
X_vectors = list(map(lambda elem: make_context_vector(elem, word2index) , X))

In [146]:
y = np.array([i[1] for i in data])
y_vectors = list(map(lambda elem: make_context_vector([elem], word2index), y))

In [165]:
X_train, X_test, y_train, y_test = train_test_split(X_vectors[:500000], y_vectors[:500000], test_size=0.2, random_state=42)

In [168]:
hotel_reviews_dataset = HotelReviewsDataset(X_train, y_train)

## 3. Modelling

In [169]:
hotel_reviews_loader = DataLoader(dataset=hotel_reviews_dataset, batch_size=BATCH_SIZE, shuffle=True)

In [179]:
class CBOW(nn.Module):
    def __init__(self, vocab_size, embedding_dim, context_size):
      super(CBOW, self).__init__()
      self.embeddings = nn.Embedding(vocab_size, embedding_dim, device=device)
      self.linear1 = nn.Linear(context_size * embedding_dim, 128)
      self.activation_function1 = nn.ReLU()
      self.linear2 = nn.Linear(128, vocab_size)
      self.activation_function2 = nn.LogSoftmax(dim=1)

    def forward(self, inputs):
      embeds = self.embeddings(inputs).view(inputs.size(0), -1)
      out = self.linear1(embeds)
      out = self.activation_function1(out)
      out = self.linear2(out)
      out = self.activation_function2(out)
      return out

    def write_embedding_to_file(self,filename):
      for i in self.embeddings.parameters():
        weights = i.data.numpy()
      np.save(filename,weights)
    
    def get_word_emdedding(self, word):
        word = torch.tensor([word_to_ix[word]])
        return self.embeddings(word).view(1,-1)

In [180]:
def train_model(model, data_loader, epochs, word2index):

  losses = np.zeros(epochs)
  loss_function = nn.NLLLoss()
  optimizer = optim.Adam(model.parameters())

  for epoch in range(epochs):
    for step, (context_vectors, target_vector) in enumerate(data_loader):

      context_vectors = context_vectors.to(device) # Move the batch of context vectors into GPU memory
      target_vector = target_vector.to(device) # Move the batch of target vectors into GPU memory 

      model.zero_grad() # Reset all gradients back to zero

      log_probs = model(context_vectors) # forward pass
      loss = loss_function(log_probs, torch.squeeze(target_vector)) # compute loss for batch
      losses[epoch] += loss.item() # accumulate loss

      loss.backward() # backpropagation
      optimizer.step() # update the model weights

    print("Epoch {0}/{1} ... Average loss {2}".format(epoch+1, epochs, losses[epoch] / len(data_loader.dataset))) # Print average loss in this episode
  return losses

In [181]:
model = CBOW(reviews_vocabulary_size, EMBEDDING_DIM_HOTEL, 2*CONTEXT_OFFSET).to(device)
losses = train_model(model, hotel_reviews_loader, EPOCHS_HOTEL, word2index)

Epoch 1/12 ... Average loss 0.1255742033147812
Epoch 2/12 ... Average loss 0.11557961870789528
Epoch 3/12 ... Average loss 0.10951060964822769
Epoch 4/12 ... Average loss 0.10464652255535126
Epoch 5/12 ... Average loss 0.10073291439414024
Epoch 6/12 ... Average loss 0.09756090621948242
Epoch 7/12 ... Average loss 0.09486423535823822
Epoch 8/12 ... Average loss 0.09255975857257843
Epoch 9/12 ... Average loss 0.0905779037463665
Epoch 10/12 ... Average loss 0.08882011162638664
Epoch 11/12 ... Average loss 0.08722553806781769
Epoch 12/12 ... Average loss 0.08578425164818763


In [ ]:
#TESTING
#context = ['People','create','to', 'direct']
#context_vector = make_context_vector(context, word_to_ix)
#a = model(context_vector)

#Print result
#print(f'Raw text: {" ".join(raw_text)}\n')
#print(f'Context: {context}\n')
#print(f'Prediction: {ix_to_word[torch.argmax(a[0]).item()]}')

In [ ]:
# Function that will be called in every epoch
def train_epoch(loss_function, optimizer, model, loader):
  
  # Keep track of the total loss for the batch
  total_loss = 0
  for batch_inputs, batch_labels, batch_lengths in loader:
    # Clear the gradients
    optimizer.zero_grad()
    # Run a forward pass
    outputs = model.forward(batch_inputs)
    # Compute the batch loss
    loss = loss_function(outputs, batch_labels, batch_lengths)
    # Calculate the gradients
    loss.backward()
    # Update the parameteres
    optimizer.step()
    total_loss += loss.item()

  return total_loss


# Function containing our main training loop
def train(loss_function, optimizer, model, loader, num_epochs=10000):

  # Iterate through each epoch and call our train_epoch function
  for epoch in range(num_epochs):
    epoch_loss = train_epoch(loss_function, optimizer, model, loader)
    if epoch % 100 == 0: print(epoch_loss)

In [ ]:


def make_context_vector(context, word_to_ix):
    idxs = [word_to_ix[w] for w in context]
    return torch.tensor(idxs, dtype=torch.long)

raw_text = """We are about to study the idea of a computational process.
Computational processes are abstract beings that inhabit computers.
As they evolve, processes manipulate other abstract things called data.
The evolution of a process is directed by a pattern of rules
called a program. People create programs to direct processes. In effect,
we conjure the spirits of the computer with our spells.""".split()


# By deriving a set from `raw_text`, we deduplicate the array
vocab = set(raw_text)
vocab_size = len(vocab)

word_to_ix = {word:ix for ix, word in enumerate(vocab)}
ix_to_word = {ix:word for ix, word in enumerate(vocab)}

data = []
for i in range(2, len(raw_text) - 2):
    context = [raw_text[i - 2], raw_text[i - 1],
               raw_text[i + 1], raw_text[i + 2]]
    target = raw_text[i]
    data.append((context, target))




#TRAINING
for epoch in range(50):
    total_loss = 0

    for context, target in data:
        context_vector = make_context_vector(context, word_to_ix)  

        log_probs = model(context_vector)

        total_loss += loss_function(log_probs, torch.tensor([word_to_ix[target]]))

    #optimize at the end of each epoch
    optimizer.zero_grad()
    total_loss.backward()
    optimizer.step()

#TESTING
context = ['People','create','to', 'direct']
context_vector = make_context_vector(context, word_to_ix)
a = model(context_vector)

#Print result
print(f'Raw text: {" ".join(raw_text)}\n')
print(f'Context: {context}\n')
print(f'Prediction: {ix_to_word[torch.argmax(a[0]).item()]}')